In [ ]:
!pip install ipython-autotime
%load_ext autotime

from google.colab import drive
import os
from datetime import datetime

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/DL and CV')

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 4.03 s (started: 2022-05-04 19:35:45 +00:00)


In [ ]:
#-- Check the running GPU
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-6d3d0e36-4f9a-1d9e-6149-cda40305eed2)
time: 192 ms (started: 2022-05-04 19:35:49 +00:00)


In [ ]:
#-- Running Tensorboard within google colab noebook
%load_ext tensorboard
%tensorboard --logdir="./models/MNIST/runs" #-- Read logs from MNIST directory
# %tensorboard --logdir="./models/CIFAR/runs" #-- Read logs from CIFAR directory

# Imports

In [ ]:
import time
import random
from tqdm.notebook import tqdm_notebook
from pathlib import Path

import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

import torch
from torch import nn

import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
from torchvision.models import googlenet
from torch.utils.tensorboard import SummaryWriter

time: 17.9 ms (started: 2022-05-04 19:35:53 +00:00)


In [ ]:
def seed():
  """ This function is used when running any cell to make sure all the seed are the same"""
  rand_seed = 0
  random.seed(rand_seed)
  os.environ['PYTHONHASHSEED'] = str(rand_seed)
  np.random.seed(rand_seed)
  torch.manual_seed(rand_seed)
  torch.cuda.manual_seed(rand_seed)
  torch.cuda.manual_seed_all(rand_seed) # if you are using multi-GPU.
  torch.backends.cudnn.benchmark = False
  torch.backends.cudnn.deterministic = True

time: 6.54 ms (started: 2022-05-04 19:35:53 +00:00)


# Train & Validate

In [ ]:
#-- Freeze the randomness
seed()

def model_train(model, model_name, lr, momentum, optimizer, epochs, tb_writer, parameters):
  #-- Build a lamba function to get the current time stamp when it is called
  ts = lambda x: x.now().strftime('%d_%m_%Y__%H:%M:%S')  
  #-- Change the GooglNet output layer to 10 to match the number of MNIST and CIFAR classes
  model.fc = nn.Linear(1024, 10)  
  #-- Pass model to to cuda
  model.to(device)  
  #-- Define the loss criterion
  loss_criterion = nn.CrossEntropyLoss()  

  print('###################### Training {} {} model for {} epochs ######################'.format(model_name, parameters, epochs))
  #-- Set the lowest validation loss to positive infinity 
  best_val_loss = np.inf
  #-- Loop over the dataset for a number of epochs 
  for epoch in range(epochs):
      #-- Initiate train loss, correct training, epoch loss, epoch accuracy, and epoch_accuracy
      train_loss = 0.0
      train_correct = 0
      total = 0 
      epoch_loss = 0.0
      epoch_acc = 0.0

      #-- Set model to train mode
      model.train()

      #-- Loop over the train loader dataset
      for i, (images, labels) in tqdm_notebook(enumerate(train_loader)): 
          #-- Pass images and labels to cuda
          images, labels = images.to(device), labels.to(device)
          #-- Empty the gradiant
          optimizer.zero_grad() 
          #-- Feed-forward pass
          outputs = model(images) 
          outputs = outputs.logits
          #-- Get the predictions 
          _, preds = torch.max(outputs, 1)  
          #-- Training loss calculation
          loss = loss_criterion(outputs, labels)  
          #-- Apply backpropagation to calculate gradients
          loss.backward()
          #-- Adjusting learning weights
          optimizer.step() 
          #-- Gather data and report
          train_loss += loss.item() * images.size(0) 
          total += labels.size(0)
          train_correct += (preds == labels).sum().item()  

          #-- Print results every 100 batch
          if (i + 1) % 100 == 0:
            print_ = 'Epoch {} of {}, Step: {} of {}, Training loss: {:.7f}, Training accuracy: {:.7f}, Time: {}'
            print(print_.format(epoch + 1, epochs, i + 1, len(train_loader), (train_loss / total), (train_correct / total), ts(datetime) ))
          epoch_loss = train_loss / total
          epoch_acc = train_correct / total

      #-- Print end of epoch training's results and save them in tensorboard
      print_ = 'Epoch {} of {}, Average training loss: {:.7f}, Average training accuracy: {:.7f}, Time: {}'
      print(print_.format(epoch + 1, epochs, epoch_loss,epoch_acc, ts(datetime)))
      tb_writer.add_scalars(model_name+'_epoch_loss_accuracy_'+parameters, {'train_loss':epoch_loss, 'train_accuracy':epoch_acc}, epoch + 1)
      
      print('###################### Validating {} {} model ######################'.format(model_name, parameters))
      
      #-- Initiate validation loss, correct validation , epoch loss, epoch accuracy, and epoch_accuracy
      val_loss = 0.0
      val_correct = 0
      total = 0
      val_epoch_loss = 0.0
      val_epoch_acc = 0.0

      with torch.no_grad(): 
        #-- Set model to evaluation mode
        model.eval()
        for i, (val_images, val_labels) in tqdm_notebook(enumerate(val_loader)):
            #-- Pass images and labels to cuda
            val_images, val_labels = val_images.to(device), val_labels.to(device)

            outputs = model(val_images)  
            #-- Get the predictions
            _, predicted = torch.max(outputs.data, 1)
            #-- Validation loss calculation
            loss = loss_criterion(outputs, val_labels)
            #-- Print results every 100 batch
            val_loss += loss.item() * val_images.size(0)  
            total += val_labels.size(0)
            val_correct += (predicted == val_labels).sum().item()
            
            #-- Check every 100 step
            if (i + 1) % 100 == 0:
              #-- if there a new loss that is ower than best_val_loss , save model
              if val_epoch_loss < best_val_loss :
                print_ = 'Step: {} of {}, Validation loss: {:.7f}, Validation accuracy: {:.7f}, Time: {}'
                print(print_.format(i + 1, len(val_loader), (val_loss / total), (val_correct / total) ,ts(datetime)), end=' | ')
                print('Loss decreased from {:.7f} to {:.7f} .... Saving the model'.format(best_val_loss, val_epoch_loss))
                best_val_loss = val_epoch_loss

                #-- Saving the model state dict
                model_name_extra = '__lr {}__momentum {}__epochs {}'.format(lr, momentum, epochs)
                model_save_path = os.path.join(model_dir, model_name+model_name_extra)
                torch.save(model.state_dict(), model_save_path)

              else:
                print_ = 'Step: {} of {}, Validation loss: {:.7f}, Validation accuracy: {:.7f}, Time: {}'
                print(print_.format(i + 1, len(val_loader), (val_loss / total), (val_correct / total),ts(datetime)))

            val_epoch_loss = val_loss / total
            val_epoch_acc = val_correct / total

        #-- Print end of epoch training's results and save them in tensorboard
        print_ = 'Average validation loss: {:.7f}, Average validation accuracy: {:.7f}, Time: {}'
        print(print_.format(val_epoch_loss, val_epoch_acc,ts(datetime)))
        tb_writer.add_scalars(model_name+'_epoch_loss_accuracy_'+parameters, {'val_loss':val_epoch_loss, 'val_accuracy':val_epoch_acc}, epoch + 1)

  return model, model_save_path


time: 273 ms (started: 2022-05-04 19:35:53 +00:00)


# Test

In [ ]:
#-- Freeze the randomness
seed()

def model_test(model, model_name, tb_writer, parameters):  
  ts = lambda x: x.now().strftime('%d_%m_%Y__%H:%M:%S')

  print('###################### Testing {} {} model ######################'.format(model_name, parameters))
  correct = 0
  total = 0
  test_pred_labels = []
  test_actual_labels = []
  incorrect_pred = 0
  correct_pred = 0

  with torch.no_grad():
    for i, (test_images, test_labels) in tqdm_notebook(enumerate(test_loader)):
      #-- Pass images and labels to Cuda
      test_images, test_labels = test_images.to(device), test_labels.to(device)
      #-- Feed-forward pass
      outputs = model(test_images)  
      #-- Get the predictions
      _, predicted = torch.max(outputs.data, 1)
      #-- Change CPU to numpy
      temp = predicted.detach().cpu().numpy()  
      #-- Add the prediction to the test_pred_labels list
      test_pred_labels.append(temp)  
      #-- Add the acutal labels to the test_actual_labels list
      test_actual_labels.append(test_labels.detach().cpu().numpy())
      total += test_labels.size(0)
      correct += (predicted == test_labels).sum().item()
      #-- Check if the ground truth labels match the predicted labels
      if ~test_labels.all().eq(predicted.all()): #-- If so, save 10 sample images
        if incorrect_pred < 10:
          plot_results(test_images, test_labels, predicted, i, model_name, parameters, 'incorrect')
          incorrect_pred +=1
      else:
        if correct_pred < 10: #-- If not, save 10 sample images
          plot_results(test_images, test_labels, predicted, i, model_name, parameters, 'correct')
          correct_pred +=1
      #-- Print results every 100 batch
      if (i + 1) % 100 == 0:
        print('Step: {} of {}, Test accuracy: {:.7f}, Time: {}'.format(i+1, len(test_loader), (correct / total), ts(datetime) ))
  
  #-- Calculate the final accuracy and then print it
  acc = correct / total
  print('Average testing accuracy: {:.7f}, Time: {}'.format(acc, ts(datetime)))
  tb_writer.add_text(model_name+'_test_accuracy_'+parameters, str(acc))


time: 76.9 ms (started: 2022-05-04 19:35:53 +00:00)


# Plot results

In [ ]:
#-- Create a graph to compare the the actual labels with the predicted results
def plot_results(test_images, test_labels, predicted, index, model_name, parameters, correct_incorrect):
  fig = plt.figure()
  fig.suptitle('Testing {} model ({}) {} \n {}'.format(model_name, index+1, parameters, time.ctime()), fontsize=13)
  for i, (image, label, pred) in enumerate(zip(test_images, test_labels, predicted)):
    plt.subplot(1,test_images.shape[0], i+1) 
    plt.tight_layout(pad=0)
    plt.imshow(image[0].cpu(), cmap='gray')
    plt.title("Actual: {} \n Prediction: {}".format(label, pred), fontsize=10)
    plt.xticks([])
    plt.yticks([])

  #-- Save images to disk and tensorboard
  tb_writer.add_figure('Testing {} model ({}) {}'.format(model_name, index+1, parameters), fig, 0)
  image_save_path = os.path.join(output_dir, 'Test {} model_{}_{}_{}.png'.format(model_name, index+1, parameters, correct_incorrect))
  fig.savefig(image_save_path, bbox_inches='tight', pad_inches=0, transparent=True)

time: 15 ms (started: 2022-05-04 19:35:54 +00:00)


# Loading and Spliting MNIST Data

In [ ]:
#-- Freeze the randomness
seed()

#-- Prepare GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#-- Create data, train, and test directories
data_dir  = './datasets/MNIST'
train_dir = os.path.join(data_dir, 'train')
test_dir  = os.path.join(data_dir, 'test')

#-- Create directories for models and outputs
model_dir  = './models/MNIST'
output_dir = '/content/drive/MyDrive/DL and CV/outputs/MNIST'
Path(output_dir).mkdir(parents=True, exist_ok=True)

trans = transforms.Compose([transforms.Resize(224), #-- GoogleNet model requires this image shape
                              transforms.RandomRotation(15), #-- images augmentation to challenge the model
                              transforms.Grayscale(num_output_channels=3), #-- GoogleNet model requires 3 channels
                              transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]) #-- Values normalization

#-- Download MNIST training data, and splitting it to training and validation
train_data = datasets.MNIST(root=train_dir, train=True, download=True, transform=trans)

#-- Split the training data to training and validation datasets.
train_indices, val_indices = train_test_split(list(range(len(train_data.targets))), test_size=5000, stratify=train_data.targets, random_state=0)
train = torch.utils.data.Subset(train_data, train_indices)
val = torch.utils.data.Subset(train_data, val_indices)

#-- Download MNIST test data
test = datasets.MNIST(root=test_dir, train=False, download=True, transform=trans)

#-- Setting training batch size:
batch_size = 4

#-- Build the training, validation, and test data loaders
train_loader = DataLoader(dataset=train, batch_size=batch_size, shuffle=False)  
val_loader   = DataLoader(dataset=val,   batch_size=batch_size, shuffle=False)  
test_loader  = DataLoader(dataset=test,  batch_size=batch_size, shuffle=False) 

#-- Print out the number of salmples for training, vlidation, and test sets.
print("Number of Samples in Training Dataset: ",   len(train))
print("Number of Samples in Validation Dataset: ", len(val))
print("Number of Samples in Testing Dataset: ",    len(test))

Number of Samples in Training Dataset:  55000
Number of Samples in Validation Dataset:  5000
Number of Samples in Testing Dataset:  10000
time: 339 ms (started: 2022-05-04 19:35:54 +00:00)


# MNIST Main

## Part 1

In [ ]:
#-- Parameters
epochs = 5

for learning_rate in [0.1, 0.01, 0.001, 0.0001]:
  for mom in [0, 0.3, 0.6, 0.9]:
    #-- Freeze the randomness
    seed()

    model = googlenet(pretrained=False)
    model_name = 'googlenet'
    lr = learning_rate
    momentum = mom
    parameters = 'SGD, lr_{}, momentum_{}'.format(lr, momentum)

    #-- Initiating a tensorboard writer that contains all logs for training, validation, and testing
    tb_writer = SummaryWriter('./models/MNIST/runs', filename_suffix='_'+model_name+'_'+parameters)

    #-- Create the model's critrion loss function and a optimization function 
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    #-- Train and valiate the model
    model, model_save_path = model_train(model, model_name, lr, momentum, optimizer, epochs, tb_writer, parameters)

    #-- Load the best saved model for testing
    model.load_state_dict(torch.load(model_save_path))
    model.to(device)

    #-- Set the model mode to evaluation to prepare for testing
    model.eval()
    model_test(model, model_name, tb_writer, parameters) 

    #-- Close the Tensoboard writer
    tb_writer.close()

    #-- Delete the model
    del model


/usr/local/lib/python3.7/dist-packages/torchvision/models/googlenet.py:50: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  FutureWarning,


###################### Training googlenet 5 model for SGD, lr_0.1, momentum_0 epochs ######################


0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Epoch 1 of 5, Step: 100 of 13750, Training loss: 2.0482917, Training accuracy: 0.3050000, Time: 04_05_2022__19:36:07
Epoch 1 of 5, Step: 200 of 13750, Training loss: 1.7179204, Training accuracy: 0.4250000, Time: 04_05_2022__19:36:16
Epoch 1 of 5, Step: 300 of 13750, Training loss: 1.5112144, Training accuracy: 0.5141667, Time: 04_05_2022__19:36:23
Epoch 1 of 5, Step: 400 of 13750, Training loss: 1.3589697, Training accuracy: 0.5793750, Time: 04_05_2022__19:36:30
Epoch 1 of 5, Step: 500 of 13750, Training loss: 1.2519677, Training accuracy: 0.6205000, Time: 04_05_2022__19:36:37
Epoch 1 of 5, Step: 600 of 13750, Training loss: 1.1653221, Training accuracy: 0.6529167, Time: 04_05_2022__19:36:44
Epoch 1 of 5, Step: 700 of 13750, Training loss: 1.0879151, Training accuracy: 0.6814286, Time: 04_05_2022__19:36:51
Epoch 1 of 5, Step: 800 of 13750, Training loss: 1.0246514, Training accuracy: 0.7025000, Time: 04_05_2022__19:36:58
Epoch 1 of 5, Step: 900 of 13750, Training loss: 0.9815927, Trai

...

Epoch 3 of 5, Step: 100 of 13750, Training loss: 0.0611767, Training accuracy: 0.9950000, Time: 05_05_2022__04:02:01
Epoch 3 of 5, Step: 200 of 13750, Training loss: 0.0576708, Training accuracy: 0.9937500, Time: 05_05_2022__04:02:08
Epoch 3 of 5, Step: 300 of 13750, Training loss: 0.0587659, Training accuracy: 0.9908333, Time: 05_05_2022__04:02:15
Epoch 3 of 5, Step: 400 of 13750, Training loss: 0.0639008, Training accuracy: 0.9868750, Time: 05_05_2022__04:02:21
Epoch 3 of 5, Step: 500 of 13750, Training loss: 0.0653376, Training accuracy: 0.9865000, Time: 05_05_2022__04:02:28
Epoch 3 of 5, Step: 600 of 13750, Training loss: 0.0687139, Training accuracy: 0.9850000, Time: 05_05_2022__04:02:35
Epoch 3 of 5, Step: 700 of 13750, Training loss: 0.0718785, Training accuracy: 0.9853571, Time: 05_05_2022__04:02:42
Epoch 3 of 5, Step: 800 of 13750, Training loss: 0.0741112, Training accuracy: 0.9843750, Time: 05_05_2022__04:02:49
Epoch 3 of 5, Step: 900 of 13750, Training loss: 0.0768061, Trai

0it [00:00, ?it/s]

Step: 100 of 1250, Validation loss: 0.0198892, Validation accuracy: 0.9950000, Time: 05_05_2022__04:17:19 | Loss decreased from 0.0230103 to 0.0199490 .... Saving the model
Step: 200 of 1250, Validation loss: 0.0256179, Validation accuracy: 0.9950000, Time: 05_05_2022__04:17:21
Step: 300 of 1250, Validation loss: 0.0227613, Validation accuracy: 0.9950000, Time: 05_05_2022__04:17:24
Step: 400 of 1250, Validation loss: 0.0215925, Validation accuracy: 0.9950000, Time: 05_05_2022__04:17:26 | Loss decreased from 0.0199490 to 0.0198058 .... Saving the model
Step: 500 of 1250, Validation loss: 0.0231803, Validation accuracy: 0.9955000, Time: 05_05_2022__04:17:29
Step: 600 of 1250, Validation loss: 0.0253559, Validation accuracy: 0.9941667, Time: 05_05_2022__04:17:31
Step: 700 of 1250, Validation loss: 0.0238180, Validation accuracy: 0.9946429, Time: 05_05_2022__04:17:34
Step: 800 of 1250, Validation loss: 0.0233871, Validation accuracy: 0.9946875, Time: 05_05_2022__04:17:36
Step: 900 of 1250,

0it [00:00, ?it/s]

Epoch 4 of 5, Step: 100 of 13750, Training loss: 0.0410446, Training accuracy: 0.9950000, Time: 05_05_2022__04:17:54
Epoch 4 of 5, Step: 200 of 13750, Training loss: 0.0399703, Training accuracy: 0.9962500, Time: 05_05_2022__04:18:01
Epoch 4 of 5, Step: 300 of 13750, Training loss: 0.0430084, Training accuracy: 0.9933333, Time: 05_05_2022__04:18:07
Epoch 4 of 5, Step: 400 of 13750, Training loss: 0.0443322, Training accuracy: 0.9937500, Time: 05_05_2022__04:18:14
Epoch 4 of 5, Step: 500 of 13750, Training loss: 0.0446692, Training accuracy: 0.9930000, Time: 05_05_2022__04:18:21
Epoch 4 of 5, Step: 600 of 13750, Training loss: 0.0509191, Training accuracy: 0.9912500, Time: 05_05_2022__04:18:28
Epoch 4 of 5, Step: 700 of 13750, Training loss: 0.0529959, Training accuracy: 0.9910714, Time: 05_05_2022__04:18:34
Epoch 4 of 5, Step: 800 of 13750, Training loss: 0.0559078, Training accuracy: 0.9890625, Time: 05_05_2022__04:18:41
Epoch 4 of 5, Step: 900 of 13750, Training loss: 0.0582141, Trai

## Part 2

In [ ]:
#-- Parameters
epochs = 5

for learning_rate in [0.01, 0.001, 0.0001]:
  for mom in [0, 0.3, 0.6, 0.9]:
    if learning_rate == 0.01 and mom == 0:
      continue
    if learning_rate == 0.01 and mom == 0.3:
      continue
    #-- Freeze the randomness
    seed()

    model = googlenet(pretrained=False)
    model_name = 'googlenet'
    lr = learning_rate
    momentum = mom
    parameters = 'SGD, lr_{}, momentum_{}'.format(lr, momentum)

    #-- Initiating a tensorboard writer that contains all logs for training, validation, and testing
    tb_writer = SummaryWriter('./models/MNIST/runs', filename_suffix='_'+model_name+'_'+parameters)

    #-- Create the model's critrion loss function and a optimization function 
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    #-- Train and valiate the model
    model, model_save_path = model_train(model, model_name, lr, momentum, optimizer, epochs, tb_writer, parameters)

    #-- Load the best saved model for testing
    model.load_state_dict(torch.load(model_save_path))
    model.to(device)

    #-- Set the model mode to evaluation to prepare for testing
    model.eval()
    model_test(model, model_name, tb_writer, parameters) 

    #-- Close the Tensoboard writer
    tb_writer.close()

    #-- Delete the model
    del model

/usr/local/lib/python3.7/dist-packages/torchvision/models/googlenet.py:50: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  FutureWarning,


###################### Training googlenet 5 model for SGD, lr_0.01, momentum_0.6 epochs ######################


0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Epoch 1 of 5, Step: 100 of 13750, Training loss: 1.8129195, Training accuracy: 0.3900000, Time: 05_05_2022__12:19:26
Epoch 1 of 5, Step: 200 of 13750, Training loss: 1.4256356, Training accuracy: 0.5700000, Time: 05_05_2022__12:19:30
Epoch 1 of 5, Step: 300 of 13750, Training loss: 1.2153658, Training accuracy: 0.6525000, Time: 05_05_2022__12:19:35
Epoch 1 of 5, Step: 400 of 13750, Training loss: 1.0875606, Training accuracy: 0.6993750, Time: 05_05_2022__12:19:40
Epoch 1 of 5, Step: 500 of 13750, Training loss: 0.9891588, Training accuracy: 0.7285000, Time: 05_05_2022__12:19:45
Epoch 1 of 5, Step: 600 of 13750, Training loss: 0.9185722, Training accuracy: 0.7508333, Time: 05_05_2022__12:19:50
Epoch 1 of 5, Step: 700 of 13750, Training loss: 0.8573064, Training accuracy: 0.7692857, Time: 05_05_2022__12:19:54
Epoch 1 of 5, Step: 800 of 13750, Training loss: 0.8008746, Training accuracy: 0.7865625, Time: 05_05_2022__12:19:59
Epoch 1 of 5, Step: 900 of 13750, Training loss: 0.7618355, Trai

...

Epoch 4 of 5, Step: 100 of 13750, Training loss: 0.1315777, Training accuracy: 0.9875000, Time: 05_05_2022__20:49:03
Epoch 4 of 5, Step: 200 of 13750, Training loss: 0.1222975, Training accuracy: 0.9862500, Time: 05_05_2022__20:49:08
Epoch 4 of 5, Step: 300 of 13750, Training loss: 0.1251944, Training accuracy: 0.9833333, Time: 05_05_2022__20:49:13
Epoch 4 of 5, Step: 400 of 13750, Training loss: 0.1319969, Training accuracy: 0.9812500, Time: 05_05_2022__20:49:18
Epoch 4 of 5, Step: 500 of 13750, Training loss: 0.1368461, Training accuracy: 0.9800000, Time: 05_05_2022__20:49:23
Epoch 4 of 5, Step: 600 of 13750, Training loss: 0.1419038, Training accuracy: 0.9787500, Time: 05_05_2022__20:49:28
Epoch 4 of 5, Step: 700 of 13750, Training loss: 0.1439111, Training accuracy: 0.9778571, Time: 05_05_2022__20:49:34
Epoch 4 of 5, Step: 800 of 13750, Training loss: 0.1439215, Training accuracy: 0.9781250, Time: 05_05_2022__20:49:38
Epoch 4 of 5, Step: 900 of 13750, Training loss: 0.1476087, Trai

0it [00:00, ?it/s]

Step: 100 of 1250, Validation loss: 0.0362135, Validation accuracy: 0.9925000, Time: 05_05_2022__21:00:35
Step: 200 of 1250, Validation loss: 0.0399835, Validation accuracy: 0.9925000, Time: 05_05_2022__21:00:37
Step: 300 of 1250, Validation loss: 0.0384496, Validation accuracy: 0.9916667, Time: 05_05_2022__21:00:39
Step: 400 of 1250, Validation loss: 0.0343506, Validation accuracy: 0.9925000, Time: 05_05_2022__21:00:42
Step: 500 of 1250, Validation loss: 0.0329734, Validation accuracy: 0.9925000, Time: 05_05_2022__21:00:44 | Loss decreased from 0.0338517 to 0.0330085 .... Saving the model
Step: 600 of 1250, Validation loss: 0.0336967, Validation accuracy: 0.9925000, Time: 05_05_2022__21:00:46
Step: 700 of 1250, Validation loss: 0.0316505, Validation accuracy: 0.9928571, Time: 05_05_2022__21:00:48 | Loss decreased from 0.0330085 to 0.0309932 .... Saving the model
Step: 800 of 1250, Validation loss: 0.0316307, Validation accuracy: 0.9931250, Time: 05_05_2022__21:00:50
Step: 900 of 1250,

0it [00:00, ?it/s]

Epoch 5 of 5, Step: 100 of 13750, Training loss: 0.1156080, Training accuracy: 0.9850000, Time: 05_05_2022__21:01:06
Epoch 5 of 5, Step: 200 of 13750, Training loss: 0.1041052, Training accuracy: 0.9900000, Time: 05_05_2022__21:01:11
Epoch 5 of 5, Step: 300 of 13750, Training loss: 0.1014451, Training accuracy: 0.9900000, Time: 05_05_2022__21:01:16
Epoch 5 of 5, Step: 400 of 13750, Training loss: 0.1079930, Training accuracy: 0.9875000, Time: 05_05_2022__21:01:21
Epoch 5 of 5, Step: 500 of 13750, Training loss: 0.1128835, Training accuracy: 0.9865000, Time: 05_05_2022__21:01:26
Epoch 5 of 5, Step: 600 of 13750, Training loss: 0.1166750, Training accuracy: 0.9858333, Time: 05_05_2022__21:01:31
Epoch 5 of 5, Step: 700 of 13750, Training loss: 0.1173621, Training accuracy: 0.9857143, Time: 05_05_2022__21:01:36
Epoch 5 of 5, Step: 800 of 13750, Training loss: 0.1176418, Training accuracy: 0.9853125, Time: 05_05_2022__21:01:41
Epoch 5 of 5, Step: 900 of 13750, Training loss: 0.1215971, Trai

## Part 3

In [ ]:
#-- Parameters
epochs = 5

for learning_rate in [0.0001]:
  for mom in [0, 0.3, 0.6, 0.9]:
    if learning_rate == 0.0001 and mom == 0:
      continue
    if learning_rate == 0.0001 and mom == 0.3:
      continue
    #-- Freeze the randomness
    seed()

    model = googlenet(pretrained=False)
    model_name = 'googlenet'
    lr = learning_rate
    momentum = mom
    parameters = 'SGD, lr_{}, momentum_{}'.format(lr, momentum)

    #-- Initiating a tensorboard writer that contains all logs for training, validation, and testing
    tb_writer = SummaryWriter('./models/MNIST/runs', filename_suffix='_'+model_name+'_'+parameters)

    #-- Create the model's critrion loss function and a optimization function 
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    #-- Train and valiate the model
    model, model_save_path = model_train(model, model_name, lr, momentum, optimizer, epochs, tb_writer, parameters)

    #-- Load the best saved model for testing
    model.load_state_dict(torch.load(model_save_path))
    model.to(device)

    #-- Set the model mode to evaluation to prepare for testing
    model.eval()
    model_test(model, model_name, tb_writer, parameters) 

    #-- Close the Tensoboard writer
    tb_writer.close()

    #-- Delete the model
    del model

/usr/local/lib/python3.7/dist-packages/torchvision/models/googlenet.py:50: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  FutureWarning,


###################### Training googlenet 5 model for SGD, lr_0.0001, momentum_0.6 epochs ######################


0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Epoch 1 of 5, Step: 100 of 13750, Training loss: 2.2991748, Training accuracy: 0.1175000, Time: 07_05_2022__00:17:45
Epoch 1 of 5, Step: 200 of 13750, Training loss: 2.2496013, Training accuracy: 0.1462500, Time: 07_05_2022__00:17:52
Epoch 1 of 5, Step: 300 of 13750, Training loss: 2.2347625, Training accuracy: 0.1608333, Time: 07_05_2022__00:17:58
Epoch 1 of 5, Step: 400 of 13750, Training loss: 2.1996885, Training accuracy: 0.1812500, Time: 07_05_2022__00:18:05
Epoch 1 of 5, Step: 500 of 13750, Training loss: 2.1700282, Training accuracy: 0.1950000, Time: 07_05_2022__00:18:11
Epoch 1 of 5, Step: 600 of 13750, Training loss: 2.1456872, Training accuracy: 0.2108333, Time: 07_05_2022__00:18:18
Epoch 1 of 5, Step: 700 of 13750, Training loss: 2.1080572, Training accuracy: 0.2296429, Time: 07_05_2022__00:18:24
Epoch 1 of 5, Step: 800 of 13750, Training loss: 2.0641122, Training accuracy: 0.2590625, Time: 07_05_2022__00:18:31
Epoch 1 of 5, Step: 900 of 13750, Training loss: 2.0269920, Trai

...

Epoch 5 of 5, Step: 100 of 13750, Training loss: 0.0714150, Training accuracy: 0.9975000, Time: 07_05_2022__02:33:59
Epoch 5 of 5, Step: 200 of 13750, Training loss: 0.0624288, Training accuracy: 0.9962500, Time: 07_05_2022__02:34:05
Epoch 5 of 5, Step: 300 of 13750, Training loss: 0.0598172, Training accuracy: 0.9950000, Time: 07_05_2022__02:34:11
Epoch 5 of 5, Step: 400 of 13750, Training loss: 0.0630614, Training accuracy: 0.9950000, Time: 07_05_2022__02:34:18
Epoch 5 of 5, Step: 500 of 13750, Training loss: 0.0654090, Training accuracy: 0.9950000, Time: 07_05_2022__02:34:24
Epoch 5 of 5, Step: 600 of 13750, Training loss: 0.0678530, Training accuracy: 0.9937500, Time: 07_05_2022__02:34:31
Epoch 5 of 5, Step: 700 of 13750, Training loss: 0.0696859, Training accuracy: 0.9935714, Time: 07_05_2022__02:34:37
Epoch 5 of 5, Step: 800 of 13750, Training loss: 0.0701199, Training accuracy: 0.9931250, Time: 07_05_2022__02:34:44
Epoch 5 of 5, Step: 900 of 13750, Training loss: 0.0740297, Trai

0it [00:00, ?it/s]

Step: 100 of 1250, Validation loss: 0.0269036, Validation accuracy: 0.9900000, Time: 07_05_2022__02:48:32
Step: 200 of 1250, Validation loss: 0.0296608, Validation accuracy: 0.9912500, Time: 07_05_2022__02:48:34
Step: 300 of 1250, Validation loss: 0.0234275, Validation accuracy: 0.9933333, Time: 07_05_2022__02:48:37 | Loss decreased from 0.0245482 to 0.0234952 .... Saving the model
Step: 400 of 1250, Validation loss: 0.0218723, Validation accuracy: 0.9943750, Time: 07_05_2022__02:48:39 | Loss decreased from 0.0234952 to 0.0218926 .... Saving the model
Step: 500 of 1250, Validation loss: 0.0256749, Validation accuracy: 0.9935000, Time: 07_05_2022__02:48:42
Step: 600 of 1250, Validation loss: 0.0253328, Validation accuracy: 0.9941667, Time: 07_05_2022__02:48:44
Step: 700 of 1250, Validation loss: 0.0229538, Validation accuracy: 0.9950000, Time: 07_05_2022__02:48:46
Step: 800 of 1250, Validation loss: 0.0225882, Validation accuracy: 0.9953125, Time: 07_05_2022__02:48:49
Step: 900 of 1250,

0it [00:00, ?it/s]

Step: 100 of 2500, Test accuracy: 0.9900000, Time: 07_05_2022__02:49:05
Step: 200 of 2500, Test accuracy: 0.9887500, Time: 07_05_2022__02:49:08
Step: 300 of 2500, Test accuracy: 0.9900000, Time: 07_05_2022__02:49:10
Step: 400 of 2500, Test accuracy: 0.9893750, Time: 07_05_2022__02:49:12
Step: 500 of 2500, Test accuracy: 0.9880000, Time: 07_05_2022__02:49:15
Step: 600 of 2500, Test accuracy: 0.9875000, Time: 07_05_2022__02:49:18
Step: 700 of 2500, Test accuracy: 0.9889286, Time: 07_05_2022__02:49:20
Step: 800 of 2500, Test accuracy: 0.9903125, Time: 07_05_2022__02:49:22
Step: 900 of 2500, Test accuracy: 0.9902778, Time: 07_05_2022__02:49:25
Step: 1000 of 2500, Test accuracy: 0.9900000, Time: 07_05_2022__02:49:27
Step: 1100 of 2500, Test accuracy: 0.9906818, Time: 07_05_2022__02:49:29
Step: 1200 of 2500, Test accuracy: 0.9912500, Time: 07_05_2022__02:49:32
Step: 1300 of 2500, Test accuracy: 0.9917308, Time: 07_05_2022__02:49:34
Step: 1400 of 2500, Test accuracy: 0.9923214, Time: 07_05_20

# Loading and Spliting CIFAR Data

In [ ]:
#-- Freeze the randomness
seed()

#-- Prepare GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#-- Create data, train, and test directories
data_dir  = './datasets/CIFAR'
train_dir = os.path.join(data_dir, 'train')
test_dir  = os.path.join(data_dir, 'test')

#-- Create directories for models and outputs
model_dir  = './models/CIFAR'
output_dir = '/content/drive/MyDrive/DL and CV/outputs/CIFAR'
Path(output_dir).mkdir(parents=True, exist_ok=True)

trans = transforms.Compose([transforms.Resize(224), #-- googlenet model requires this image shape
                              transforms.RandomRotation(15), #-- images augmentation to challenge the model
                              transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) #-- Values normalization

#-- Download CIFAR10 training data, and splitting it to training and validation
train_data = datasets.CIFAR10(root=train_dir, train=True, download=True, transform=trans)

#-- Split the training data to training and validation datasets.
train_indices, val_indices = train_test_split(list(range(len(train_data.targets))), test_size=5000, stratify=train_data.targets, random_state=0)
train = torch.utils.data.Subset(train_data, train_indices)
val = torch.utils.data.Subset(train_data, val_indices)

#-- Download CIFAR10 test data
test = datasets.CIFAR10(root=test_dir, train=False, download=True, transform=trans)

#-- Setting training batch size:
batch_size = 4

#-- Build the training, validation, and test data loaders
train_loader = DataLoader(dataset=train, batch_size=batch_size, shuffle=False)  
val_loader   = DataLoader(dataset=val,   batch_size=batch_size, shuffle=False)  
test_loader  = DataLoader(dataset=test,  batch_size=batch_size, shuffle=False) 

print("Number of Samples in Training Dataset: ",   len(train))
print("Number of Samples in Validation Dataset: ", len(val))
print("Number of Samples in Testing Dataset: ",    len(test))

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./datasets/CIFAR/train/cifar-10-python.tar.gz to ./datasets/CIFAR/train


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./datasets/CIFAR/test/cifar-10-python.tar.gz to ./datasets/CIFAR/test
Number of Samples in Training Dataset:  45000
Number of Samples in Validation Dataset:  5000
Number of Samples in Testing Dataset:  10000
time: 20.3 s (started: 2022-05-04 19:38:01 +00:00)


# CIFAR Main

## Part 1

In [ ]:
#-- Parameters
epochs = 5

for learning_rate in [0.1, 0.01, 0.001, 0.0001]:
  for mom in [0, 0.3, 0.6, 0.9]:
    #-- Freeze the randomness
    seed()

    model = googlenet(pretrained=False)
    model_name = 'googlenet'
    lr = learning_rate
    momentum = mom
    parameters = 'SGD, lr_{}, momentum_{}'.format(lr, momentum)

    #-- Initiating a tensorboard writer that contains all logs for training, validation, and testing
    tb_writer = SummaryWriter('/models/CIFAR/runs', filename_suffix='_'+model_name+'_'+parameters)

    #-- Create the model's critrion loss function and a optimization function 
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    #-- Train and valiate the model
    model, model_save_path = model_train(model, model_name, lr, momentum, optimizer, epochs, tb_writer, parameters)

    #-- Load the best saved model for testing
    model.load_state_dict(torch.load(model_save_path))
    model.to(device)

    #-- Set the model mode to evaluation to prepare for testing
    model.eval()
    model_test(model, model_name, tb_writer, parameters) 

    #-- Close the Tensoboard writer
    tb_writer.close()

    #-- Delete the model
    del model



/usr/local/lib/python3.7/dist-packages/torchvision/models/googlenet.py:50: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  FutureWarning,


###################### Training googlenet 5 model for SGD, lr_0.1, momentum_0 epochs ######################


0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Epoch 1 of 5, Step: 100 of 11250, Training loss: 2.3430556, Training accuracy: 0.0875000, Time: 04_05_2022__19:38:47
Epoch 1 of 5, Step: 200 of 11250, Training loss: 2.3010667, Training accuracy: 0.1175000, Time: 04_05_2022__19:38:53
Epoch 1 of 5, Step: 300 of 11250, Training loss: 2.2812849, Training accuracy: 0.1366667, Time: 04_05_2022__19:39:00
Epoch 1 of 5, Step: 400 of 11250, Training loss: 2.2785286, Training accuracy: 0.1287500, Time: 04_05_2022__19:39:06
Epoch 1 of 5, Step: 500 of 11250, Training loss: 2.2685094, Training accuracy: 0.1395000, Time: 04_05_2022__19:39:12
Epoch 1 of 5, Step: 600 of 11250, Training loss: 2.2543346, Training accuracy: 0.1429167, Time: 04_05_2022__19:39:18
Epoch 1 of 5, Step: 700 of 11250, Training loss: 2.2492258, Training accuracy: 0.1485714, Time: 04_05_2022__19:39:25
Epoch 1 of 5, Step: 800 of 11250, Training loss: 2.2311238, Training accuracy: 0.1575000, Time: 04_05_2022__19:39:31
Epoch 1 of 5, Step: 900 of 11250, Training loss: 2.2220652, Trai

...

Step: 100 of 2500, Test accuracy: 0.7075000, Time: 04_05_2022__23:48:11
Step: 200 of 2500, Test accuracy: 0.7125000, Time: 04_05_2022__23:48:13
Step: 300 of 2500, Test accuracy: 0.7116667, Time: 04_05_2022__23:48:15
Step: 400 of 2500, Test accuracy: 0.7200000, Time: 04_05_2022__23:48:18
Step: 500 of 2500, Test accuracy: 0.7200000, Time: 04_05_2022__23:48:20
Step: 600 of 2500, Test accuracy: 0.7170833, Time: 04_05_2022__23:48:22
Step: 700 of 2500, Test accuracy: 0.7167857, Time: 04_05_2022__23:48:25
Step: 800 of 2500, Test accuracy: 0.7175000, Time: 04_05_2022__23:48:27
Step: 900 of 2500, Test accuracy: 0.7163889, Time: 04_05_2022__23:48:29
Step: 1000 of 2500, Test accuracy: 0.7155000, Time: 04_05_2022__23:48:32
Step: 1100 of 2500, Test accuracy: 0.7204545, Time: 04_05_2022__23:48:34
Step: 1200 of 2500, Test accuracy: 0.7227083, Time: 04_05_2022__23:48:36
Step: 1300 of 2500, Test accuracy: 0.7236538, Time: 04_05_2022__23:48:39
Step: 1400 of 2500, Test accuracy: 0.7219643, Time: 04_05_20

0it [00:00, ?it/s]

Epoch 1 of 5, Step: 100 of 11250, Training loss: 2.2969418, Training accuracy: 0.1525000, Time: 04_05_2022__23:49:13
Epoch 1 of 5, Step: 200 of 11250, Training loss: 2.2475578, Training accuracy: 0.1700000, Time: 04_05_2022__23:49:20
Epoch 1 of 5, Step: 300 of 11250, Training loss: 2.2221184, Training accuracy: 0.1808333, Time: 04_05_2022__23:49:26
Epoch 1 of 5, Step: 400 of 11250, Training loss: 2.1868126, Training accuracy: 0.1887500, Time: 04_05_2022__23:49:32
Epoch 1 of 5, Step: 500 of 11250, Training loss: 2.1609843, Training accuracy: 0.2000000, Time: 04_05_2022__23:49:38
Epoch 1 of 5, Step: 600 of 11250, Training loss: 2.1363943, Training accuracy: 0.2120833, Time: 04_05_2022__23:49:45
Epoch 1 of 5, Step: 700 of 11250, Training loss: 2.1302926, Training accuracy: 0.2139286, Time: 04_05_2022__23:49:51
Epoch 1 of 5, Step: 800 of 11250, Training loss: 2.1134468, Training accuracy: 0.2196875, Time: 04_05_2022__23:49:57
Epoch 1 of 5, Step: 900 of 11250, Training loss: 2.1005815, Trai

KeyboardInterrupt: ignored

time: 4h 21min 46s (started: 2022-05-04 19:38:21 +00:00)


## Part 2


In [ ]:
#-- Parameters
epochs = 5

for learning_rate in [0.01, 0.001, 0.0001]:
  for mom in [0, 0.3, 0.6, 0.9]:
    #-- Freeze the randomness
    seed()

    model = googlenet(pretrained=False)
    model_name = 'googlenet'
    lr = learning_rate
    momentum = mom
    parameters = 'SGD, lr_{}, momentum_{}'.format(lr, momentum)

    #-- Initiating a tensorboard writer that contains all logs for training, validation, and testing
    tb_writer = SummaryWriter('/models/CIFAR/runs', filename_suffix='_'+model_name+'_'+parameters)

    #-- Create the model's critrion loss function and a optimization function 
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    #-- Train and valiate the model
    model, model_save_path = model_train(model, model_name, lr, momentum, optimizer, epochs, tb_writer, parameters)

    #-- Load the best saved model for testing
    model.load_state_dict(torch.load(model_save_path))
    model.to(device)

    #-- Set the model mode to evaluation to prepare for testing
    model.eval()
    model_test(model, model_name, tb_writer, parameters) 

    #-- Close the Tensoboard writer
    tb_writer.close()

    #-- Delete the model
    del model


/usr/local/lib/python3.7/dist-packages/torchvision/models/googlenet.py:50: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  FutureWarning,


###################### Training googlenet 5 model for SGD, lr_0.01, momentum_0 epochs ######################


0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Epoch 1 of 5, Step: 100 of 11250, Training loss: 2.2969418, Training accuracy: 0.1525000, Time: 05_05_2022__00:02:52
Epoch 1 of 5, Step: 200 of 11250, Training loss: 2.2475578, Training accuracy: 0.1700000, Time: 05_05_2022__00:03:00
Epoch 1 of 5, Step: 300 of 11250, Training loss: 2.2221184, Training accuracy: 0.1808333, Time: 05_05_2022__00:03:07
Epoch 1 of 5, Step: 400 of 11250, Training loss: 2.1868126, Training accuracy: 0.1887500, Time: 05_05_2022__00:03:15
Epoch 1 of 5, Step: 500 of 11250, Training loss: 2.1609843, Training accuracy: 0.2000000, Time: 05_05_2022__00:03:22
Epoch 1 of 5, Step: 600 of 11250, Training loss: 2.1363943, Training accuracy: 0.2120833, Time: 05_05_2022__00:03:30
Epoch 1 of 5, Step: 700 of 11250, Training loss: 2.1302926, Training accuracy: 0.2139286, Time: 05_05_2022__00:03:37
Epoch 1 of 5, Step: 800 of 11250, Training loss: 2.1134468, Training accuracy: 0.2196875, Time: 05_05_2022__00:03:45
Epoch 1 of 5, Step: 900 of 11250, Training loss: 2.1005815, Trai

...

Epoch 4 of 5, Step: 100 of 11250, Training loss: 0.7347475, Training accuracy: 0.7600000, Time: 05_05_2022__08:24:23
Epoch 4 of 5, Step: 200 of 11250, Training loss: 0.7938291, Training accuracy: 0.7437500, Time: 05_05_2022__08:24:30
Epoch 4 of 5, Step: 300 of 11250, Training loss: 0.8198792, Training accuracy: 0.7416667, Time: 05_05_2022__08:24:38
Epoch 4 of 5, Step: 400 of 11250, Training loss: 0.8100746, Training accuracy: 0.7506250, Time: 05_05_2022__08:24:46
Epoch 4 of 5, Step: 500 of 11250, Training loss: 0.8186375, Training accuracy: 0.7475000, Time: 05_05_2022__08:24:53
Epoch 4 of 5, Step: 600 of 11250, Training loss: 0.8251528, Training accuracy: 0.7445833, Time: 05_05_2022__08:25:01
Epoch 4 of 5, Step: 700 of 11250, Training loss: 0.8414091, Training accuracy: 0.7392857, Time: 05_05_2022__08:25:09
Epoch 4 of 5, Step: 800 of 11250, Training loss: 0.8447643, Training accuracy: 0.7393750, Time: 05_05_2022__08:25:17
Epoch 4 of 5, Step: 900 of 11250, Training loss: 0.8468631, Trai

0it [00:00, ?it/s]

Step: 100 of 1250, Validation loss: 0.6612797, Validation accuracy: 0.7925000, Time: 05_05_2022__08:38:47 | Loss decreased from 0.6665564 to 0.6664857 .... Saving the model
Step: 200 of 1250, Validation loss: 0.6411272, Validation accuracy: 0.7950000, Time: 05_05_2022__08:38:50 | Loss decreased from 0.6664857 to 0.6417546 .... Saving the model
Step: 300 of 1250, Validation loss: 0.6713359, Validation accuracy: 0.7900000, Time: 05_05_2022__08:38:53
Step: 400 of 1250, Validation loss: 0.6620875, Validation accuracy: 0.7850000, Time: 05_05_2022__08:38:56
Step: 500 of 1250, Validation loss: 0.6482192, Validation accuracy: 0.7900000, Time: 05_05_2022__08:39:00
Step: 600 of 1250, Validation loss: 0.6372155, Validation accuracy: 0.7950000, Time: 05_05_2022__08:39:03 | Loss decreased from 0.6417546 to 0.6381344 .... Saving the model
Step: 700 of 1250, Validation loss: 0.6327728, Validation accuracy: 0.7989286, Time: 05_05_2022__08:39:06 | Loss decreased from 0.6381344 to 0.6324996 .... Saving 

0it [00:00, ?it/s]

Epoch 5 of 5, Step: 100 of 11250, Training loss: 0.6433395, Training accuracy: 0.8025000, Time: 05_05_2022__08:39:31
Epoch 5 of 5, Step: 200 of 11250, Training loss: 0.6900677, Training accuracy: 0.7800000, Time: 05_05_2022__08:39:39
Epoch 5 of 5, Step: 300 of 11250, Training loss: 0.6928291, Training accuracy: 0.7891667, Time: 05_05_2022__08:39:47
Epoch 5 of 5, Step: 400 of 11250, Training loss: 0.6884957, Training accuracy: 0.7893750, Time: 05_05_2022__08:39:54
Epoch 5 of 5, Step: 500 of 11250, Training loss: 0.7032535, Training accuracy: 0.7855000, Time: 05_05_2022__08:40:02
Epoch 5 of 5, Step: 600 of 11250, Training loss: 0.7085085, Training accuracy: 0.7833333, Time: 05_05_2022__08:40:10
Epoch 5 of 5, Step: 700 of 11250, Training loss: 0.7147788, Training accuracy: 0.7842857, Time: 05_05_2022__08:40:17
Epoch 5 of 5, Step: 800 of 11250, Training loss: 0.7166775, Training accuracy: 0.7850000, Time: 05_05_2022__08:40:25
Epoch 5 of 5, Step: 900 of 11250, Training loss: 0.7161675, Trai

## Part 3

In [ ]:
#-- Parameters
epochs = 5

for learning_rate in [0.001, 0.0001]:
  for mom in [0, 0.3, 0.6, 0.9]:
    if learning_rate == 0.001 and mom == 0:
      continue
    if learning_rate == 0.001 and mom == 0.3:
      continue
    #-- Freeze the randomness
    seed()

    model = googlenet(pretrained=False)
    model_name = 'googlenet'
    lr = learning_rate
    momentum = mom
    parameters = 'SGD, lr_{}, momentum_{}'.format(lr, momentum)

    #-- Initiating a tensorboard writer that contains all logs for training, validation, and testing
    tb_writer = SummaryWriter('/models/CIFAR/runs', filename_suffix='_'+model_name+'_'+parameters)

    #-- Create the model's critrion loss function and a optimization function 
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    #-- Train and valiate the model
    model, model_save_path = model_train(model, model_name, lr, momentum, optimizer, epochs, tb_writer, parameters)

    #-- Load the best saved model for testing
    model.load_state_dict(torch.load(model_save_path))
    model.to(device)

    #-- Set the model mode to evaluation to prepare for testing
    model.eval()
    model_test(model, model_name, tb_writer, parameters) 

    #-- Close the Tensoboard writer
    tb_writer.close()

    #-- Delete the model
    del model


/usr/local/lib/python3.7/dist-packages/torchvision/models/googlenet.py:50: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  FutureWarning,


###################### Training googlenet 5 model for SGD, lr_0.001, momentum_0.6 epochs ######################


0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Epoch 1 of 5, Step: 100 of 11250, Training loss: 2.2739874, Training accuracy: 0.1325000, Time: 05_05_2022__12:11:52
Epoch 1 of 5, Step: 200 of 11250, Training loss: 2.2259866, Training accuracy: 0.1500000, Time: 05_05_2022__12:11:57
Epoch 1 of 5, Step: 300 of 11250, Training loss: 2.1974075, Training accuracy: 0.1716667, Time: 05_05_2022__12:12:01
Epoch 1 of 5, Step: 400 of 11250, Training loss: 2.1675890, Training accuracy: 0.1850000, Time: 05_05_2022__12:12:07
Epoch 1 of 5, Step: 500 of 11250, Training loss: 2.1403980, Training accuracy: 0.2000000, Time: 05_05_2022__12:12:12
Epoch 1 of 5, Step: 600 of 11250, Training loss: 2.1114230, Training accuracy: 0.2058333, Time: 05_05_2022__12:12:16
Epoch 1 of 5, Step: 700 of 11250, Training loss: 2.1087400, Training accuracy: 0.2071429, Time: 05_05_2022__12:12:21
Epoch 1 of 5, Step: 800 of 11250, Training loss: 2.0902098, Training accuracy: 0.2140625, Time: 05_05_2022__12:12:25
Epoch 1 of 5, Step: 900 of 11250, Training loss: 2.0778945, Trai

...

Epoch 5 of 5, Step: 100 of 11250, Training loss: 0.6919621, Training accuracy: 0.8125000, Time: 05_05_2022__16:44:09
Epoch 5 of 5, Step: 200 of 11250, Training loss: 0.7479077, Training accuracy: 0.7825000, Time: 05_05_2022__16:44:13
Epoch 5 of 5, Step: 300 of 11250, Training loss: 0.7567208, Training accuracy: 0.7808333, Time: 05_05_2022__16:44:18
Epoch 5 of 5, Step: 400 of 11250, Training loss: 0.7427267, Training accuracy: 0.7818750, Time: 05_05_2022__16:44:23
Epoch 5 of 5, Step: 500 of 11250, Training loss: 0.7565766, Training accuracy: 0.7790000, Time: 05_05_2022__16:44:28
Epoch 5 of 5, Step: 600 of 11250, Training loss: 0.7595188, Training accuracy: 0.7754167, Time: 05_05_2022__16:44:32
Epoch 5 of 5, Step: 700 of 11250, Training loss: 0.7671123, Training accuracy: 0.7728571, Time: 05_05_2022__16:44:37
Epoch 5 of 5, Step: 800 of 11250, Training loss: 0.7719671, Training accuracy: 0.7715625, Time: 05_05_2022__16:44:42
Epoch 5 of 5, Step: 900 of 11250, Training loss: 0.7728621, Trai

0it [00:00, ?it/s]

Step: 100 of 1250, Validation loss: 0.6547718, Validation accuracy: 0.7875000, Time: 05_05_2022__16:53:02
Step: 200 of 1250, Validation loss: 0.6606386, Validation accuracy: 0.7762500, Time: 05_05_2022__16:53:04
Step: 300 of 1250, Validation loss: 0.6836301, Validation accuracy: 0.7683333, Time: 05_05_2022__16:53:06
Step: 400 of 1250, Validation loss: 0.6584603, Validation accuracy: 0.7787500, Time: 05_05_2022__16:53:08
Step: 500 of 1250, Validation loss: 0.6532975, Validation accuracy: 0.7840000, Time: 05_05_2022__16:53:10 | Loss decreased from 0.6531764 to 0.6510243 .... Saving the model
Step: 600 of 1250, Validation loss: 0.6415892, Validation accuracy: 0.7891667, Time: 05_05_2022__16:53:12 | Loss decreased from 0.6510243 to 0.6425862 .... Saving the model
Step: 700 of 1250, Validation loss: 0.6303552, Validation accuracy: 0.7921429, Time: 05_05_2022__16:53:15 | Loss decreased from 0.6425862 to 0.6299696 .... Saving the model
Step: 800 of 1250, Validation loss: 0.6156790, Validation

0it [00:00, ?it/s]

Step: 100 of 2500, Test accuracy: 0.8075000, Time: 05_05_2022__16:53:35
Step: 200 of 2500, Test accuracy: 0.7925000, Time: 05_05_2022__16:53:37
Step: 300 of 2500, Test accuracy: 0.7950000, Time: 05_05_2022__16:53:39
Step: 400 of 2500, Test accuracy: 0.7931250, Time: 05_05_2022__16:53:41
Step: 500 of 2500, Test accuracy: 0.7995000, Time: 05_05_2022__16:53:43
Step: 600 of 2500, Test accuracy: 0.7950000, Time: 05_05_2022__16:53:45
Step: 700 of 2500, Test accuracy: 0.7921429, Time: 05_05_2022__16:53:47
Step: 800 of 2500, Test accuracy: 0.7943750, Time: 05_05_2022__16:53:49
Step: 900 of 2500, Test accuracy: 0.7944444, Time: 05_05_2022__16:53:51
Step: 1000 of 2500, Test accuracy: 0.7925000, Time: 05_05_2022__16:53:53
Step: 1100 of 2500, Test accuracy: 0.7956818, Time: 05_05_2022__16:53:55
Step: 1200 of 2500, Test accuracy: 0.7950000, Time: 05_05_2022__16:53:57
Step: 1300 of 2500, Test accuracy: 0.7951923, Time: 05_05_2022__16:53:59
Step: 1400 of 2500, Test accuracy: 0.7933929, Time: 05_05_20